In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tdqm
!pip install ipywidgets==7.7.1

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=3c68ad9e6502ba645e541f0a6e228db8fa1732aad766daaa9b0a5bf5bb6ff231
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.9 MB/s eta 0:00:00


In [ ]:
import requests
from tqdm import tqdm
import time
import json

# URL to fetch data from PubMed
output_file_path = "/content/drive/MyDrive/pmidtotext/PUBMED_ID_CHEMICALNAMES_TEXTS_TAG_NORM1C.json"
failed_pmid_file_path = "/content/drive/MyDrive/pmidtotext/1C_merged_pubmed_ids_fails.txt"

# Dictionary to store text associated with each PMID
pmid_texts = [{}]

with open(failed_pmid_file_path, 'w', encoding='utf-8') as failed_file:
    # Initialize tqdm to track progress
    with open('/content/drive/MyDrive/pmidtotext/1C_merged_pubmed_ids.txt', 'r', encoding='latin-1') as infile:
        lines = infile.readlines()
        total_lines = len(lines)
        progress_bar = tqdm(total=total_lines)

        start_time = time.time()
        for pmid in lines:
            pmid = pmid.strip()
            url_template = f"https://www.ncbi.nlm.nih.gov/research/pubtator3-api/publications/export/biocjson?pmids={pmid}&full=true"
            response = requests.get(url_template)

            if response.status_code == 200:
                # Parse JSON content
                try:
                    data = response.json()
                    text_accumulator = ''
                    for passage in data["passages"]:
                        text_element = passage['text'].strip()

                        if text_element is not None:
                            # Sort annotations by their offset
                            annotations = sorted(passage['annotations'], key=lambda x: x['locations'][0]['offset'])
                            offset_adjustment = 0

                            for annotation in annotations:
                                anno_type = annotation['infons']['type']
                                anno_loc = annotation['locations']
                                if annotation['infons']['normalized_id'] is not None:
                                    normal_name =  annotation['infons']['name']
                                else:
                                    normal_name = '1'

                                for loc in anno_loc:
                                    if normal_name.isnumeric():
                                        annoStart = loc["offset"] - passage["offset"]
                                        annoEnd = annoStart + loc["length"]

                                        # Insert annotations into text_element
                                        text_element = (text_element[:annoStart + offset_adjustment] + f"<{anno_type}>" +
                                                        text_element[annoStart + offset_adjustment:annoEnd + offset_adjustment] + f"</{anno_type}>" +
                                                        text_element[annoEnd + offset_adjustment:])
                                        # Update offset adjustment
                                        offset_adjustment += len(f"<{anno_type}>") + len(f"</{anno_type}>")
                                    else:
                                        annoStart = loc["offset"] - passage["offset"]
                                        annoEnd = annoStart + loc["length"]
                                        middle = len(text_element[annoStart + offset_adjustment:annoEnd + offset_adjustment])
                                        # Insert annotations into text_element
                                        text_element = (text_element[:annoStart + offset_adjustment] + f"<{anno_type}>" +
                                                        normal_name + f"</{anno_type}>" +
                                                        text_element[annoEnd + offset_adjustment:])
                                        # Update offset adjustment
                                        offset_adjustment += len(f"<{anno_type}>") + len(f"</{anno_type}>") + len(normal_name) - middle

                            text_accumulator += text_element + ' '  # Concatenate text

                    pmid_texts[0][pmid] = text_accumulator
                except json.JSONDecodeError as e:
                    print(f"Failed to parse JSON response for PMID: {pmid}")
                    print(f"Response text: {response.text}")
            else:
                print(f"Failed to retrieve data for PMID: {pmid}")
                failed_file.write(pmid + '\n')

            # Periodically dump data to JSON file
            if len(pmid_texts[0]) % 100 == 0:
                with open(output_file_path, 'w', encoding='utf-8') as outfile:
                    json.dump(pmid_texts, outfile, indent=4)

            # Update progress bar
            progress_bar.update(1)

            # Limit requests to 3 per second
            if progress_bar.n % 3 == 0:
                elapsed_time = time.time() - start_time
                if elapsed_time < 1:
                    time.sleep(1 - elapsed_time)
                start_time = time.time()

        progress_bar.close()

# Dump the remaining data to the JSON file
with open(output_file_path, 'w', encoding='utf-8') as outfile:
    json.dump(pmid_texts, outfile, indent=4)


  0%|          | 33/40228 [00:14<3:04:07,  3.64it/s]

Failed to retrieve data for PMID: 32859779


  0%|          | 42/40228 [00:17<2:43:35,  4.09it/s]

Failed to retrieve data for PMID: 32606520


  0%|          | 106/40228 [00:39<3:15:44,  3.42it/s]

...

100%|█████████▉| 40208/40228 [5:33:40<00:06,  3.30it/s]

Failed to retrieve data for PMID: 32985521


100%|█████████▉| 40223/40228 [5:33:46<00:01,  3.39it/s]

Failed to retrieve data for PMID: 36459632


100%|██████████| 40228/40228 [5:33:47<00:00,  2.01it/s]


In [ ]:
import requests
import json
from pprint import pprint

url_template = f"https://www.ncbi.nlm.nih.gov/research/pubtator3-api/publications/export/biocjson?pmids={29355051}&full=true"
response = requests.get(url_template)

if response.status_code == 200:
    # Parse JSON content
    data = response.json()
    text_accumulator = ''

    for passage in data["passages"]:
        text_element = passage['text'].strip()

        if text_element is not None:
          # Sort annotations by their offset
          annotations = sorted(passage['annotations'], key=lambda x: x['locations'][0]['offset'])
          offset_adjustment = 0

          for annotation in annotations:
              anno_type = annotation['infons']['type']
              anno_loc = annotation['locations']
              if annotation['infons']['normalized_id'] is not None:
                normal_name =  annotation['infons']['name']
              else:
                  normal_name = '1'


              for loc in anno_loc:
                if normal_name.isnumeric():
                  annoStart = loc["offset"] - passage["offset"]
                  annoEnd = annoStart + loc["length"]

                  # Insert annotations into text_element
                  text_element = (text_element[:annoStart + offset_adjustment] + f"<{anno_type}>" +
                                  text_element[annoStart + offset_adjustment:annoEnd + offset_adjustment] + f"</{anno_type}>" +
                                  text_element[annoEnd + offset_adjustment:])
                  # Update offset adjustment
                  offset_adjustment += len(f"<{anno_type}>") + len(f"</{anno_type}>")
                else:
                  annoStart = loc["offset"] - passage["offset"]
                  annoEnd = annoStart + loc["length"]
                  middle = len(text_element[annoStart + offset_adjustment:annoEnd + offset_adjustment])
                  # Insert annotations into text_element
                  text_element = (text_element[:annoStart + offset_adjustment] + f"<{anno_type}>" +
                                  normal_name + f"</{anno_type}>" +
                                  text_element[annoEnd + offset_adjustment:])
                  # Update offset adjustment
                  offset_adjustment += len(f"<{anno_type}>") + len(f"</{anno_type}>") + len(normal_name) - middle

          text_accumulator += text_element + ' '  # Concatenate text

    pprint(text_accumulator)


('Fraction From <Species>Lycium barbarum</Species> '
 '<Chemical>Polysaccharides</Chemical> Reduces '
 '<Disease>Immunotoxicity</Disease> and Enhances Antitumor Activity of '
 '<Chemical>Doxorubicin</Chemical> in <Species>Mice</Species>. The aim of the '
 'present study was to investigate whether fraction from Lycium barbarum '
 'polysaccharide (LBP) could reduce <Disease>immunotoxicity</Disease> and '
 'enhance antitumor activity of <Chemical>Doxorubicin</Chemical> '
 '(<Chemical>Doxorubicin</Chemical>) in <Species>mice</Species>. A '
 '<Chemical>Water</Chemical>-soluble LBP fraction, designated LBP3, was '
 'isolated from edible <Chemical>Chinese herbal</Chemical> <Species>Lycium '
 'barbarum</Species> and used in this study. To investigate the effect of LBP3 '
 'on <Chemical>Doxorubicin</Chemical>-induced '
 '<Disease>immunotoxicity</Disease>, <Disease>Neoplasms</Disease>-free '
 '<Species>mice</Species> were used and treated with either normal saline, '
 '<Chemical>Doxorubicin</Che